# Оценка линейного искажающего оператора в задаче восстановления изображений

In [ ]:
import numpy as np
from skimage import color, data, restoration, img_as_float, io
# import matplotlib.pyplot as plt
from scipy.signal import convolve2d as conv2

# from skimage import color, data, restoration
# from skimage.restoration import uft
# from scipy.signal import fftconvolve, convolve, convolve2d
# from scipy.stats.stats import pearsonr
# from numpy.fft import fftn, ifftn
from skimage.measure import compare_psnr
# from skimage.draw import bezier_curve
# import scipy.ndimage.filters as fi
# from itertools import chain
# from skimage.draw import line_aa
# from math import sin, cos, pi
from functools import partial
from scipy.optimize import minimize

from util import *
from richardson_lucy import *

In [ ]:
astro = color.rgb2gray(data.astronaut())
liftingbody = img_as_float(io.imread('liftingbody.png'))
#liftingbody = edgetaper(liftingbody, gkern2(41,6))

In [ ]:
sz = 11
psf = np.ones((sz, sz)) / (sz*sz)
astro_noisy = conv2(astro, psf, 'same')
# Add Noise to Image
#astro_noisy += (np.random.poisson(lam=25, size=astro.shape) - 10) / 255.

In [ ]:
# Restore Image using Richardson-Lucy algorithm:
iterations = 7
deconv = richardson_lucy_matlab(astro_noisy, psf, iterations=iterations, eps=1e-5)
show_results(astro, astro_noisy, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
lifting_blurred = conv2(liftingbody, psf, 'same')

In [ ]:
# Restore Image using Richardson-Lucy algorithm my:
iterations = 40
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, useFFT=False)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

## Линейный оператор смаза

In [ ]:
np.round(motion_blur_psf_my(x=3,y=-4.5),3)

In [ ]:
shift = 30
psf = motion_blur_psf_my(shift, pi/3)
lifting_blurred = conv2(liftingbody, psf, 'same')
iterations = 12
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=0.004)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])
correlation_X = pearsonr(liftingbody.ravel('C')[:-1], liftingbody.ravel('C')[1:])
correlation_Y = pearsonr(liftingbody.ravel('F')[:-1], liftingbody.ravel('F')[1:])
print(correlation_X, correlation_Y)

## Неверная psf

In [ ]:
psf_wrong = motion_blur_psf(shift, pi/4)
#iterations = 20
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

## Использование параметра dampar

In [ ]:
s_n, S_find = find_noise(liftingbody)
s_n

In [ ]:
np.sum(motion_blur_psf(shift, pi/4))

In [ ]:
psf = motion_blur_psf(shift, pi/4)
x0_ref = [shift * np.cos(pi/4), shift * np.sin(pi/4)]
lifting_blurred = conv2(liftingbody, psf, 'same')
#lifting_blurred = edgetaper(lifting_blurred, taper_psf)
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=s_n, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

In [ ]:
psf = motion_blur_psf(shift, pi/4)
x0_ref = [shift * np.cos(pi/4), shift * np.sin(pi/4)]
lifting_blurred = conv2(liftingbody, psf, 'same')
#lifting_blurred = edgetaper(lifting_blurred, taper_psf)
iterations = 30
deconv = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, eps=1e-5, clip=True, dampar=s_n, useFFT=True)
show_results(liftingbody, lifting_blurred, deconv['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY']])

### dampar и неверная psf

In [ ]:
psf_wrong = motion_blur_psf_my(shift, pi/3)
#iterations = 40
deconv_wrong = richardson_lucy_matlab(lifting_blurred, psf_wrong, iterations=iterations, eps=1e-5, dampar=s_n)
show_results(liftingbody, lifting_blurred, deconv_wrong['image'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_wrong['correlationX'],
                         deconv_wrong['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(wrong psf)', 'column correlation(wrong psf)'])

```
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter


w, h = lifting_blurred.shape
x = np.arange(w)
y = np.arange(h)
X, Y = np.meshgrid(x, y)

fig = plt.figure(figsize=(10,10))
ax = fig.gca(projection='3d')
# Plot the surface.
surf = ax.plot_surface(X, Y, F, cmap=cm.coolwarm,
                       linewidth=0, antialiased=False)

# Customize the z axis.
#ax.set_zlim(0, 1.01)
#ax.zaxis.set_major_locator(LinearLocator(10))
#ax.zaxis.set_major_formatter(FormatStrFormatter('%.02f'))

# Add a color bar which maps values to colors.
#fig.colorbar(surf, shrink=0.5, aspect=5)

plt.show()
```

## Кепстр
$$K = F^{-1}\{log(1+\left|F\{I\}\right|)\}$$

In [ ]:
N,M = lifting_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = [nn // n - n//2, nn % n - m//2]
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

## Уточнение искажающего оператора
$$\varepsilon = ||\tilde{I}\oplus\tilde{h}-I_0|| \to \min_{(x,y)}$$
Пусть $$\tilde{I}\oplus(\tilde{h}+\tilde{dh})=I_0$$
Тогда $$\tilde{I}\oplus\tilde{dh}=I_0-\tilde{I}\oplus\tilde{h}$$
Получим задачу аналогичную исходной($\tilde{I}\oplus h +\eta = I_0$)

```python
img_diff = liftingbody - convolve2d(deconv['image'], psf, mode='same') # Утечка
deconv_psf = richardson_lucy_matlab(img_diff, deconv['image'], iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
psf_new = deconv_psf['image']
deconv_upd = richardson_lucy_matlab(lifting_blurred, psf_new, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
show_results(lifting_blurred, deconv['image'], deconv_upd['image'],
             titles=['blurred', 'restored', 'restored with\nnew psf'])
plot_corr(iterations+1, [deconv['correlationX'], 
                         deconv['correlationY'],
                         deconv_upd['correlationX'],
                         deconv_upd['correlationY']], 
          legend=['row correlation', 'column correlation', 'row correlateion(new psf)', 'column correlation(new psf)'])
```

Такой подход не сработал :( Будем использовать метод Ньютона
## Уточнение искажающего оператора

$$(x,y)^T_{n+1} = (x, y)^T_{n} - H^{-1}(f(x,y))\nabla f(x,y)$$
$$H(f) =$$

In [ ]:
def funcToMinimize(xy, I_blurred, crop=50, *args, **kwargs):
    psf = motion_blur_psf_my(x=xy[0], y=xy[1])
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))

In [ ]:
partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([10,10])

In [ ]:
cos(pi/4)*30

### Минимизация методом Нелдера-Мида (симплекс-метод)
Method Nelder-Mead uses the Simplex algorithm [1], [2]. This algorithm is robust in many applications. However, if numerical computation of derivative can be trusted, other algorithms using the first and/or second derivatives information might be preferred for their better performance in general.

Method Powell is a modification of Powell’s method [3], [4] which is a conjugate direction method. It performs sequential one-dimensional minimizations along each vector of the directions set (direc field in options and info), which is updated at each iteration of the main minimization loop. The function need not be differentiable, and no derivatives are taken.

Method CG uses a nonlinear conjugate gradient algorithm by Polak and Ribiere, a variant of the Fletcher-Reeves method described in [5] pp. 120-122. Only the first derivatives are used.

Method BFGS uses the quasi-Newton method of Broyden, Fletcher, Goldfarb, and Shanno (BFGS) [5] pp. 136. It uses the first derivatives only. BFGS has proven good performance even for non-smooth optimizations. This method also returns an approximation of the Hessian inverse, stored as hess_inv in the OptimizeResult object.

Method Newton-CG uses a Newton-CG algorithm [5] pp. 168 (also known as the truncated Newton method). It uses a CG method to the compute the search direction. See also TNC method for a box-constrained minimization with a similar algorithm. Suitable for large-scale problems.

Method dogleg uses the dog-leg trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and Hessian; furthermore the Hessian is required to be positive definite.

Method trust-ncg uses the Newton conjugate gradient trust-region algorithm [5] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems.

Method trust-krylov uses the Newton GLTR trust-region algorithm [14], [15] for unconstrained minimization. This algorithm requires the gradient and either the Hessian or a function that computes the product of the Hessian with a given vector. Suitable for large-scale problems. On indefinite problems it requires usually less iterations than the trust-ncg method and is recommended for medium and large-scale problems.

Method trust-exact is a trust-region method for unconstrained minimization in which quadratic subproblems are solved almost exactly [13]. This algorithm requires the gradient and the Hessian (which is not required to be positive definite). It is, in many situations, the Newton method to converge in fewer iteraction and the most recommended for small and medium-size problems.

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='nelder-mead',
               options={'xtol': 1e-3, 'disp': True})
```

```
from scipy.optimize import minimize

res = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='powell',
               options={'xtol': 1e-3, 'disp': True})
```

## Метод Ньютона

In [ ]:
res_tnc = minimize(partial(funcToMinimize, I_blurred=lifting_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
               x0=x0, method='TNC')
print(res_tnc)

In [ ]:
res_tnc['x']

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_blurred, psf, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr_crop(liftingbody, deconv_orig['image'], 20)

psf_init = motion_blur_psf_my(x=x0[0], y=x0[1])
deconv_init = richardson_lucy_matlab(lifting_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr_crop(liftingbody, deconv_init['image'], 20)

x_found, y_found = res_tnc['x']
psf_found = motion_blur_psf_my(x=x_found, y=y_found)
deconv_found = richardson_lucy_matlab(lifting_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr_crop(liftingbody, deconv_found['image'], 20)

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, x0_ref), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_tnc['x'])])

## Оценка криволинейного оператора смаза, представленного кривой Безье

In [ ]:
sz = 20
deg = 10
points_real = [(deg-1)*sz//deg-1, sz//deg, sz-1, sz-1]
#points_real = [sz, 0, sz, sz]
psf_bezier = bezier_psf2(points_real)
plt.imshow(psf_bezier)
plt.show()

In [ ]:
points_real

In [ ]:
lifting_bezier_blurred = convolve2d(liftingbody, psf_bezier, 'same')

lifting_restored = restoration.richardson_lucy(lifting_bezier_blurred, psf_bezier, iterations=iterations)
psnr_py = compare_psnr_crop(liftingbody, lifting_restored)

deconv_bezier = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, clip=True, useFFT=True, dampar=0.004)
psnr_my = compare_psnr_crop(liftingbody, deconv_bezier['image'])

In [ ]:
show_results(lifting_bezier_blurred, lifting_restored, deconv_bezier['image'],
            titles=['Blurred image', 'Restored with python LR\nPSNR={0}'.format(psnr_py), 'Restored with my LR\nPSNR={0}'.format(psnr_my)])

### Кепстр

In [ ]:
s_n, S_find = find_noise(lifting_bezier_blurred)

In [ ]:
b_clip = 3
N,M = lifting_bezier_blurred.shape
# ( ifft2 (100* log (1+ abs ( fft2 ( I ) ) ) ) ) ;
K = np.fft.ifftn(100*np.log(1+np.abs(np.fft.fftn(lifting_bezier_blurred))))
K_shift = np.fft.fftshift(K)

In [ ]:
nn = np.argmin(np.real(K_shift))
n, m, = K_shift.shape
x0 = np.array([nn // n - n//2, nn % n - m//2])
print(x0)

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(np.clip(np.real(K_shift),-1,1))#, vmin=noisy.min(), vmax=noisy.max())
plt.plot(n//2 + x0[0], m//2 + x0[1], 'ro')
plt.title('Кепстр изображения с выделенным минмиумом')
plt.savefig('kepstr.png')
plt.show()

In [ ]:
#x0 = np.concatenate((x0/3, x0))
#print(x0)

### Минимизация

In [ ]:
def funcToMinimizeCurved(xy, I_blurred, crop=100, lambd=1e-4, *args, **kwargs):
    xy = np.array(xy)
    r1 = np.sqrt(np.sum(xy[:2]**2))
    r2 = np.sqrt(np.sum((xy[2:]-xy[:2])**2))
    psf = bezier_psf2(xy, n=100)
    restored = richardson_lucy_matlab(I_blurred, psf, *args, **kwargs)
    I_restored = restored['image']
    df = convolve2d(I_restored, psf, 'same', 'symm') - I_blurred
    return np.mean(np.square(df[crop:-crop, crop:-crop]))# + lambd / (min(r1,r2)+1e-6)
partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)([0, 10, 10, 10])

In [ ]:
x0

In [ ]:
res_cg = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True),
                   options={'eps':1}, x0=x0, method='CG')
print(res_cg)

In [ ]:
nn, mm = 20, 20
step = 3
a = np.ones((nn, mm))
f = partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
for i in range(nn//step):
    for j in range(mm//step):
        xx = np.array([i*step, j*step, 19, 19])
        a[i, j] = f(xx)
        print(i*step, j*step, a[i, j])

In [ ]:
b = a[:nn//step,:mm//step]

In [ ]:
ind = np.argmin(b)
c = ind // b.shape[1]
r = ind % b.shape[1]
print(c, r)
plt.plot(r,c, 'ro')

plt.imshow(b, 'gray', vmax = 6.642956293716207e-05)
plt.title('Cost function for second point coordinates\ndarker is better')
plt.show()

In [ ]:
partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)(points_real)

In [ ]:
nn, mm = 20, 20
box= 3
a = np.ones((nn, mm))
f = partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True)
for ii, i in enumerate(range(r-box,r+box+1)):
    for jj,j in enumerate(range(c-box,c+box+1)):
        xx = np.array([i, j, 19,19])
        a[jj, ii] = f(xx)
        print(i, j, a[j, i])

In [ ]:
near = a[:2*box+1,:2*box+1]

In [ ]:
ind = np.argmin(near)
cc = ind // near.shape[1]
rr = ind % near.shape[1]
print(cc, rr)
plt.plot(rr,cc,'ro')

plt.imshow(near, 'gray')
plt.title('Goal function value for second point coordinates\ndarker is better')
plt.show()

In [ ]:
near.max(), near.min(), near.max()-near.min()

In [ ]:
ccc, rrr = c*4+cc, r*4+rr

In [ ]:
c*4+6, r*4+1

In [ ]:
points_found = [ccc, rrr, 19, 19]
psf_bezier_found = bezier_psf2(points_found)

In [ ]:
plt.imshow(psf_bezier_found)
plt.show()

In [ ]:
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

#points_found = res_basin['x']
#psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, points_found)])

In [ ]:
points_found = res_tnc['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_tnc['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_tnc['x'])])

In [ ]:
#x0 = [12, 10, 20, 20]
res_nelder = minimize(partial(funcToMinimizeCurved, I_blurred=lifting_bezier_blurred, iterations=iterations, eps=1e-5, dampar=s_n, useFFT=True), x0=x0, method='Nelder-Mead', options={'eps':1, 'xtol': 1e-3, 'disp': True})
print(res_nelder)

In [ ]:
deconv_orig = richardson_lucy_matlab(lifting_bezier_blurred, psf_bezier, iterations=iterations, dampar=s_n, useFFT=False)
psnr_orig = compare_psnr(liftingbody, deconv_orig['image'])

psf_init = bezier_psf2(x0)
deconv_init = richardson_lucy_matlab(lifting_bezier_blurred, psf_init, iterations=iterations, dampar=s_n, useFFT=False)
psnr_init = compare_psnr(liftingbody, deconv_init['image'])

points_found = res_nelder['x']
psf_found = bezier_psf2(points_found)
deconv_found = richardson_lucy_matlab(lifting_bezier_blurred, psf_found, iterations=iterations, dampar=s_n, useFFT=False)
psnr_found = compare_psnr(liftingbody, deconv_found['image'])

show_results(deconv_orig['image'], deconv_init['image'], deconv_found['image'],
             titles=['Restored with true psf\nPSNR={0}\ncoord={1}'.format(psnr_orig, points_real), 
                     'With initial approxiamtion\nPSNR={0}\ncoord={1}'.format(psnr_init, x0),
                     'Minimized error\nPSNR={0}\ncoord={1}'.format(psnr_found, res_nelder['x'])])

* Запустить градиентный метод
* Сделать мощный edgetaper
* Сделать антиалиасинг psf